In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_validate

In [19]:
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['figure.dpi']= 150

In [20]:
data = pd.read_csv('../data.csv')
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


In [21]:
data = data[data["banner_id0"] == data["banner_id"]]

In [22]:
data['g0'] = abs(data['g0'])
data['g1'] = abs(data['g1'])

In [23]:
data.drop(["campaign_clicks"], axis=1, inplace=True)

In [24]:
data

,date_time,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1
5,2021-09-27 00:06:50.000000,5,5,2377014068362699676,2,2,5,0.004,0.337634,-3.222757,37,0.004,0.338195,-3.221755,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15821466,2021-10-01 20:46:50.000000,14,254,315650865902704335,0,5,254,0.010,0.049660,-4.066035,418,0.010,0.051728,-4.052198,1,0
15821467,2021-10-02 15:51:35.000000,146,530,4329496688011613719,2,9,530,0.020,0.040037,-3.495224,481,0.020,0.039634,-3.493091,1,0
15821469,2021-10-02 17:41:10.000000,12,1236,9112780675655118328,2,0,1236,0.046,0.021433,-5.986478,1234,0.046,0.021363,-5.986109,1,0
15821470,2021-09-29 00:39:32.000000,967,21,6968514095695555037,0,0,21,0.014,0.083901,-3.025758,2,0.014,0.083865,-3.063858,1,0


In [25]:
train_data = data.drop(["oaid_hash", "rate0", "rate1", "g0", "g1", "coeff_sum0", "coeff_sum1"], axis=1)

In [26]:
categorical_features = ["zone_id", "banner_id", "os_id", "country_id"]

In [27]:
# def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
#     FITLERING_FRACTION = 0.05
#     RELABEL = 100000000000
#     size = len(data)
#     threshold = size * FITLERING_FRACTION
    
#     for field in categorical_features:
#         if field in ['banner_id', 'oaid_hash']:
#             continue
#         print(field)
#         uniques = np.unique(data[field])
#         mapping = {i: i for i in uniques}
#         for val in uniques:
#             ids = data[field] == val 
#             if np.sum(ids) < threshold:
#                 mapping[val] = RELABEL
#         data[field] = data[field].map(mapping)
    
#     for field in categorical_features:
#         print(field)
#         uniques = np.unique(data[field])
#         mapping = {v: i for i, v in enumerate(uniques)}
#         data[field] = data[field].map(mapping)
    
#     for field in categorical_features:
#         one_hot = pd.get_dummies(data[field])
#         data = data.drop(field,axis = 1)
#         data = data.join(one_hot, lsuffix=f"{field}_")
#     return data

def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    FITLERING_FRACTION = 0.05
    RELABEL = 100000000000
    size = len(data)
    threshold = size * FITLERING_FRACTION
    
    for field in categorical_features:
        if field == "banner_id":
            continue
        print(field)
        uniques = np.unique(data[field])
        mapping = {i: i for i in uniques}
        for val in uniques:
            ids = data[field] == val 
            if np.sum(ids) < threshold:
                mapping[val] = RELABEL
        data[field] = data[field].map(mapping)

    for field in categorical_features:
        one_hot = pd.get_dummies(data[field])
        data = data.drop(field,axis = 1)
        data = data.join(one_hot, lsuffix=f"{field}_")
    return data

In [ ]:
train_data = feature_engineering(train_data)

zone_id
os_id
country_id


In [ ]:
train_data.columns

In [36]:
max(data["date_time"])

'2021-10-02 23:59:59.000000'

In [37]:
max_date = "2021-10-02"

In [38]:
last_day_data = data[data.date_time.apply(lambda d: max_date in d)]

In [39]:
last_day_train_data = train_data[train_data.date_time.apply(lambda d: max_date in d)]

In [40]:
train_data.drop(last_day_train_data.index, inplace=True)

In [41]:
data.drop(["date_time"], axis=1, inplace=True)
train_data.drop(["date_time"], axis=1, inplace=True)
last_day_data.drop(["date_time"], axis=1, inplace=True)
last_day_train_data.drop(["date_time"], axis=1, inplace=True)

/tmp/ipykernel_543631/1329581662.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_day_data.drop(["date_time"], axis=1, inplace=True)
/tmp/ipykernel_543631/1329581662.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_day_train_data.drop(["date_time"], axis=1, inplace=True)


In [42]:
def create_model(data: pd.DataFrame):
    classifier = SGDClassifier(loss="modified_huber", alpha=0.1)
    return classifier

In [43]:
X = train_data.drop(["clicks"], axis=1)
y = train_data["clicks"]

In [44]:
clf = create_model(data)

In [45]:
clf.fit(X, y)

SGDClassifier(alpha=0.1, loss='modified_huber')

In [46]:
X_test = last_day_train_data.drop(["clicks"], axis=1)
y_test = last_day_train_data["clicks"]

In [47]:
y_pred = clf.predict_proba(X_test)[:, 1]

In [48]:
import scipy

$$
P(Y < X) = P(Y - X < 0) = F_{Y - X}(0)
$$

$X, Y$ заданы нормальными распределениями, их разность $Z = Y - X$ тоже со следующими коэффициентами: $Z \sim N(coeff_1 - coeff_2, g_0^2 + g_1^2)$

PDF для такого распределения в точке 0: 
$$F_Z(0) = \Phi\left(\frac{coeff_0 - coeff_1}{\sqrt{g_0^2 + g_1^2}}\right)$$

In [49]:
X_test

,zone_id,banner_id,os_id,country_id,impressions
164,2,76,1,2,1
166,0,46,4,0,1
168,2,76,2,2,1
169,0,46,0,0,1
359,0,2,0,0,1
...,...,...,...,...,...
15821452,2,89,3,1,1
15821455,2,132,2,0,1
15821461,1,52,2,3,1
15821467,2,529,2,6,1
